### Imports

In [313]:
import logging
import random
from functools import reduce

logging.getLogger().setLevel(logging.INFO)

### Define the class

In [314]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i*2+1 for i in range(num_rows)]
        self._k = k

    def nimming_remove(self, row: int, num_objects: int) -> None:
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects < self._k
        self._rows[row] -= num_objects
    
    def nimming_add(self, row: int, num_objects: int) -> None:
        self._rows[row] += num_objects

    def goal(self) -> bool:
        # Check if someone has won
        return sum(self._rows) == 0

### Random player

In [315]:
class RandomPlayer:
    def __init__(self) -> None:
        self._moves = []
        self._nMoves = 0
    
    def play(self, nim: Nim) -> None:
        # Chose a random row and a random number of pieces
        x = random.randint(0, len(nim._rows)-1)
        while nim._rows[x] == 0:
            x = random.randint(0, len(nim._rows)-1)
        y = random.randint(1, nim._rows[x])
        # Update the attributes
        self._nMoves += 1
        self._moves.append(f"{y} items from row {x}")
        nim.nimming_remove(x, y)
        
    def printSolution(self) -> None:
        logging.info(f" Random player won, moves have been: {self._moves}; total: {self._nMoves}")

### Task3.1: An agent using fixed rules based on nim-sum (i.e., an expert system)

In [316]:
def verify_state(nrow: int, num_objects: int, nim: Nim) -> bool:
    # Verify of the new game state is useful for the player with the nim sum rule
    xor = reduce(lambda a, b: a ^ b, nim._rows)
    if xor == 0:
        return True
    else:
        return False

In [317]:
class ExpertSystem:
    def __init__(self) -> None:
        self._nMoves = 0
        self._moves = []

    def printSolution(self) -> None:
        logging.info(f" Expert system won, moves have been: {self._moves}; total: {self._nMoves}")
    
    def play(self, nim: Nim) -> None:
        x = 0
        y = 0
        valid = False
        flag = True
        not_found = False
        x_already_taken = []
        # Chose a random row and a random number of pieces
        while flag:
            x = random.randint(0, len(nim._rows)-1)
            # If I have already tried the row x
            while nim._rows[x] == 0 or x in x_already_taken:
                if len(x_already_taken) == len(nim._rows):
                    # All rows have already been tried
                    flag = False
                    not_found = True
                    break
                x = random.randint(0, len(nim._rows)-1)
            x_already_taken.append(x)
            y_already_taken = []
            while flag == True and valid == False and len(y_already_taken) < nim._rows[x]:
                y = random.randint(1, nim._rows[x])
                if y not in y_already_taken:
                    # If the deleting of y objects has not been already tried
                    y_already_taken.append(y)
                    nim.nimming_remove(x, y)
                    valid = verify_state(x, y, nim)
                    if valid == True:
                        # Found a good move
                        flag = False
                    else:
                        # Restore the game as before if the solution is useless for the player (search for another)
                        nim.nimming_add(x, y)

        if not_found:
            # If no valid moves were found, do a random move
            x = random.randint(0, len(nim._rows)-1)
            while nim._rows[x] == 0:
                x = random.randint(0, len(nim._rows)-1)
            y = random.randint(1, nim._rows[x])
            nim.nimming_remove(x, y)
            
        self._nMoves += 1
        self._moves.append(f"{y} items from row {x}")

In [318]:
nim = Nim(4)
player1 = RandomPlayer()
player2 = ExpertSystem()
win = random.choice([1, 2])
print("Start game: ", nim._rows)
while nim.goal() == False:
    if win == 1:
        player1.play(nim)
        print("Game after Random player played: ", nim._rows)
        win = 2
    else:
        player2.play(nim)
        print("Game after Expert system played: ", nim._rows)
        win = 1
if win == 2:
    player1.printSolution()
else:
    player2.printSolution()

INFO:root: Expert system won, moves have been: ['2 items from row 3', '3 items from row 1', '5 items from row 2']; total: 3


Start game:  [1, 3, 5, 7]
Game after Expert system played:  [1, 3, 5, 5]
Game after Random player played:  [0, 3, 5, 5]
Game after Expert system played:  [0, 0, 5, 5]
Game after Random player played:  [0, 0, 5, 0]
Game after Expert system played:  [0, 0, 0, 0]
